In [ ]:
!pip install boto3

In [7]:
import os
import boto3
import json
import requests

import sys
sys.path.append(os.getcwd())

# Utilitary build functions

# Information for communication protocols

In [8]:
# Account
account_id = '060004687794'

# Server
region = "sa-east-1"

# Platform
ecr_image_name = "serverless-example"
tag='latest'

# API
endpoint = "predict"
method_verb='POST'
stage = "test"

# Ellaborate information
tagged_image_uri=f"{ecr_image_name}:latest"
password_stdin=f"{account_id}.dkr.ecr.{region}.amazonaws.com"

# Specify the role name and trust policy for the Lambda service
role_name = 'lambda-exec-role'

trust_policy = {
    'Version': '2012-10-17',
    'Statement': [
        {
            'Effect': 'Allow',
            'Principal': {'Service': 'lambda.amazonaws.com'},
            'Action': 'sts:AssumeRole'
        }
    ]
}

policy_arn = 'arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'

# Rate limits: Harsh since this will be public facing
# Quota: Low daily limits for the same reason
usage_constraints = {
    'rate_limits': {
        'burstLimit': 10,
        'rateLimit': 10.0
    },
    'quota': {
        'limit': 100,
        'period': 'DAY'
    }
}


In [20]:
# Function description
func_description='SKLearn predict Lambda function'

# Function name (not public facing)
function_name = f'lambda-fn-{ecr_image_name}'

# Clients

In [9]:
# Set up the IAM client
iam_client = boto3.client('iam', region_name=region)

# Set up the Lambda client
lambda_client = boto3.client('lambda', region_name=region)

# Set up the API Gateway client
gateway_client = boto3.client('apigateway', region_name=region)

# Development steps

- IAM role image handling;
- ECR image
- Lambda function creation;
- API Gateway

## IAM Role Image Handling

The first step: create a user on IAM with below permissions:

- **IAMUserChangePassword**: a default permission to change password 
- **IAMFullAccess**: Allows IAM management
- **AmazonEC2ContainerRegistryFullAccess**: Allows uploading image to ECR
- **AWSLambda_FullAccess**: Allows access to specific Lambda function given a role 
- **AmazonAPIGatewayAdministrator**: Allows access to specific API Gateway handling 



In [21]:
from deploy_utils.iam_utils import try_attach_role_policy

# The id "role_arn" will be used on lambda deployment
role_arn = try_attach_role_policy(iam_client, role_name, policy_arn, trust_policy)

role_arn

'arn:aws:iam::060004687794:role/lambda-exec-role'

## ECR image

Run 2 cells below with key stroke _Shift+Enter_ to upload the docker image to ECR.

In [22]:
from deploy_utils.ecr_utils import pipe_push_image

pipe_push_image(account_id, region, ecr_image_name, tag)

WARNING! Your password will be stored unencrypted in /home/brunolnetto/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:sa-east-1:060004687794:repository/serverless-example",
        "registryId": "060004687794",
        "repositoryName": "serverless-example",
        "repositoryUri": "060004687794.dkr.ecr.sa-east-1.amazonaws.com/serverless-example",
        "createdAt": "2023-09-16T14:56:15-03:00",
        "imageTagMutability": "MUTABLE"
    }
}
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:sa-east-1:060004687794:repository/serverless-example",
        "registryId": "060004687794",
        "repositoryName": "serverless-example",
        "repositoryUri": "060004687794.dkr.ecr.sa-east-1.amazonaws.com/serverless-example",
        "createdAt": "2023-09-16T15:24:36-03:00",
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": true
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}
sha256:b31fa18356792bb29f465c4bd8

# Lambda function creation

Run 2 cells below with key stroke _Shift+Enter_ to generate a Lambda Function based on Docker Image

### Create function

In [13]:
from deploy_utils.ecr_utils import build_ecr_url
from deploy_utils.lambda_utils import delete_function, create_function

# Retrieve (if already exists) or create a new Lambda function
routed_url = build_ecr_url(account_id, region, ecr_image_name, tag)
deletion_response = delete_function(lambda_client, function_name)
create_response = create_function(lambda_client, function_name, func_description, routed_url, role_arn)

create_response

{'ResponseMetadata': {'RequestId': '9edb43f1-16d4-431c-9c60-f40fc077458c',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Sat, 16 Sep 2023 17:58:55 GMT',
   'content-type': 'application/json',
   'content-length': '1111',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9edb43f1-16d4-431c-9c60-f40fc077458c'},
  'RetryAttempts': 0},
 'FunctionName': 'lambda-fn-serverless-example',
 'FunctionArn': 'arn:aws:lambda:sa-east-1:060004687794:function:lambda-fn-serverless-example',
 'Role': 'arn:aws:iam::060004687794:role/lambda-exec-role',
 'CodeSize': 0,
 'Description': 'SKLearn predict Lambda function',
 'Timeout': 10,
 'MemorySize': 256,
 'LastModified': '2023-09-16T17:58:54.837+0000',
 'CodeSha256': '1c4edb51807ff77ec42f238ebb533299495ffa05c1d216d14b7fee5f902052f8',
 'Version': '27',
 'TracingConfig': {'Mode': 'PassThrough'},
 'RevisionId': 'c2914916-a07a-4ea5-81d9-54e2113c1f07',
 'State': 'Pending',
 'StateReason': 'The function is being created.',
 'StateReasonCode': 'Creating

### Get function

In [15]:
from deploy_utils.lambda_utils import get_function

get_response = get_function(lambda_client, function_name)

get_response

{'ResponseMetadata': {'RequestId': '0a1a93ac-bea9-4ffc-87a2-cf4959593dac',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 16 Sep 2023 17:59:40 GMT',
   'content-type': 'application/json',
   'content-length': '1427',
   'connection': 'keep-alive',
   'x-amzn-requestid': '0a1a93ac-bea9-4ffc-87a2-cf4959593dac'},
  'RetryAttempts': 0},
 'Configuration': {'FunctionName': 'lambda-fn-serverless-example',
  'FunctionArn': 'arn:aws:lambda:sa-east-1:060004687794:function:lambda-fn-serverless-example',
  'Role': 'arn:aws:iam::060004687794:role/lambda-exec-role',
  'CodeSize': 0,
  'Description': 'SKLearn predict Lambda function',
  'Timeout': 10,
  'MemorySize': 256,
  'LastModified': '2023-09-16T17:58:54.837+0000',
  'CodeSha256': '1c4edb51807ff77ec42f238ebb533299495ffa05c1d216d14b7fee5f902052f8',
  'Version': '$LATEST',
  'TracingConfig': {'Mode': 'PassThrough'},
  'RevisionId': '43d69de9-a17b-477c-8b91-befee9dbdff1',
  'State': 'Active',
  'LastUpdateStatus': 'Successful',
  'Packag

### Test function

In [16]:
from json import dumps, loads

# Prepare the event to pass to the Lambda function
example=[1, 2, 3, 4, 5]

# Transform into json format
payload=dumps({"body": example})

# Invoke the Lambda function
response = lambda_client.invoke(
    FunctionName=function_name,
    InvocationType='RequestResponse',
    Payload=payload
)

# Get the response from the Lambda function
result = loads(response['Payload'].read())

print(result)

{'errorMessage': "Unable to import module 'lambda_api.predict_service': No module named 'lambda_api'", 'errorType': 'Runtime.ImportModuleError', 'requestId': '4c0d8b50-a247-4a54-bcbf-799c50bf9d60', 'stackTrace': []}


## API Gateway setup

Run the cells below to set the Lambda Function as an Endpoint on API Gateway.

In [ ]:
from deploy_utils.api_gateway_utils import deploy_rest_api, build_api_url

# Define the name of the API (not public facing)
rest_api_name = function_name + '-api'

deployment_reponse = deploy_rest_api(\
    gateway_client, lambda_client, \
    account_id, region, \
    function_name, rest_api_name, endpoint, method_verb, \
    usage_constraints, stage \
)

deployment_reponse

In [ ]:
rest_api_id=deployment_reponse['rest_api_id']

# The URL by default will follow this pattern:
api_url = build_api_url(rest_api_id, region, endpoint, stage)

print(api_url)


In [ ]:
api_key=deployment_reponse['api_key']
rest_api_id=deployment_reponse['rest_api_id']

# The URL by default will follow this pattern:
api_url = build_api_url(rest_api_id, region, endpoint, stage)

print(api_key)
print(api_url)


In [ ]:
# Prepare the event to pass to the Lambda function
example=[1, 2, 3, 4, 5]

# Transform into json format
payload=json.dumps({"body": example})

headers = {
    'Content-type': 'application/json', 
    'x-api-key': api_key,
}

resp = requests.post(api_url, headers=headers, json=example)
resp.json()
